<a href="https://colab.research.google.com/github/nnilayy/Detectron2/blob/main/Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DETECTRON2 IMPLEMENTATION 

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!nvidia-smi

Wed Dec 21 17:57:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

IMPORTING DEPENDENCIES

In [137]:
import detectron2 
import cv2
import multiprocessing as mp
import os
from detectron2 import model_zoo
from detectron2.config import get_cfg
from google.colab.patches import cv2_imshow
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog, DatasetCatalog

FILE STRUCTURE FOR THE MODEL

In [ ]:
# TRAIN SET
train_name="train"
train_path="img/train"
train_annotations="train.json"
register_coco_instances(train_name,{},train_annotation,train_path)

# TEST SET
test_name="test"
test_path="img/test"
test_annotations="test.json"
register_coco_instances(test_name,{},test_annotation,test_path)

# MENTIONING THE CLASSES
classes=['Homer', 'Marge', 'Maggie', 'Lisa', 'Bart'] 

In [ ]:
# FOR DIFFERENT TYPES OF MODELS GO TO THE FOLLOWING SITE
# https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md#license


# ----------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------
# IMAGE SEGMENTATION MODELS
model_seg=[
#(0)R50-C4: 
"COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x.yaml",

#(1)R50-DC5: 
"COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x.yaml",

#(2)R50-FPN: 
"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml",

#(3)R50-C4:	
"COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml",

#(4)R50-DC5: 
"COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml",	

#(5)R50-FPN: 
"COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",	

#(6)R101-C4: 
"COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x.yaml",

#(7)R101-DC5: 
"COCO-InstanceSegmentation/mask_rcnn_R_101_DC5_3x.yaml",	

#(8)R101-FPN:	
"COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml",

#(9)X101-FPN: 
"COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
]
# ----------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------



ASSIGNING THE MODEL

In [ ]:
model=model_seg[0]

TRAINING THE MODEL

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.MODEL.WEIGHTS=model_zoo.get_checkpoint_url(model)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.DATASETS.TRAIN = (train_name)
cfg.DATASETS.TEST = (test_name)

cfg.SOLVER.MAX_ITER = 600
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.SOLVER.BASE_LR = 0.001
cfg.TEST.EVAL_PERIOD = 200

cfg.DATALOADER.NUM_WORKERS= mp.cpu_count()
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.MODEL.DEVICE ="cuda" # cpu
cfg.OUTPUT_DIR="/content/outputModels"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(True)
trainer.train()

TESTING & VISUALIZING THE MODEL

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.WEIGHTS="outputModels/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE='cuda' #cpu

In [ ]:
predictor=DefaultPredictor(cfg) 

In [ ]:
metadata=MetadataCatalog.get(train_name).set(thing_classes=classes)
test_data = DatasetCatalog.get(test_name)

In [ ]:
for test_img in test_data:
    img = cv2.imread(test_data["file_name"])
    outputs = predictor(img)
    
    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata, 
        scale=0.8, 
        instance_mode=ColorMode.IMAGE_BW
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

TENSORBOARD LOSS CURVES AND METRICS

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

REFERENCES 
(1)
(2)
(3)
(4)

(1)TAKE CARE OF WEIGHTS FILE